In [1]:
# !sudo apt install gcc -y

# !pip install catboost
# !pip install pandas
# !pip install -U scikit-learn
# !pip install seaborn
# !pip install pyarrow
# !pip install transformers
# !pip install setfit
# !pip install torch
# !pip install xformers
# !pip install py-stringmatching

In [2]:
!nvidia-smi

Tue May 30 18:01:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
|  0%   54C    P8    14W / 170W |      1MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import json
from functools import partial
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

from transformers import pipeline

from sklearn.model_selection import KFold

In [4]:
features = pd.read_parquet("denis_features2.parquet")
features

,target,variantid1,variantid2,name1,categories1,color_parsed1,pic_embeddings_resnet_v11,main_pic_embeddings_resnet_v11,name_bert_641,characteristic_attributes_mapping1,...,cat3_grouped,text,deberta_label_0,deberta_score_0,deberta_label_1,deberta_score_1,deberta_label_2,deberta_score_2,deberta_label_4,deberta_score_4
0,0.0,51197862,51198054,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...","[-0.5104684, 0.56158644, 0.58873796, -0.529718...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500...",...,"Сетевые фильтры, разветвители и удлинители",Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303...,0,0.97,0,0.97,0,0.97,0,0.97
1,0.0,51197862,51199884,Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...","[-0.5104684, 0.56158644, 0.58873796, -0.529718...","{""Число жил"":[""3""],""Макс. нагрузка, Вт"":[""3500...",...,"Сетевые фильтры, разветвители и удлинители",Удлинитель TDM Electric Люкс УЛ05В 5 м (SQ1303...,0,0.97,0,0.97,0,0.97,0,0.97
2,1.0,53062686,536165289,Картридж лазерный Комус 729 (4370B002) черный ...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Расходн...","[черный, чер]",None,"[[-0.0032773763, 0.32531193, -0.33156675, 0.41...","[-0.53706163, 0.37264067, 0.44363657, -0.37289...","{""Количество в упаковке, шт"":[""1""],""Бренд"":[""К...",...,Расходник для печати,Картридж лазерный Комус 729 (4370B002) черный ...,0,0.79,0,0.86,0,0.83,0,0.74
3,1.0,53602615,587809782,Картридж лазерный Комус 729 (4368B002) пурпурн...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Расходн...",[пурпурный],None,"[[0.027616128, 0.33428708, -0.37326592, 0.4108...","[-0.61162275, 0.4953002, 0.47400212, -0.429568...","{""Назначение"":[""для лазерного принтера""],""Совм...",...,Расходник для печати,Картридж лазерный Комус 729 (4368B002) пурпурн...,0,0.81,0,0.84,0,0.82,0,0.80
4,1.0,53602615,615149925,Картридж лазерный Комус 729 (4368B002) пурпурн...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Расходн...",[пурпурный],None,"[[0.027616128, 0.33428708, -0.37326592, 0.4108...","[-0.61162275, 0.4953002, 0.47400212, -0.429568...","{""Назначение"":[""для лазерного принтера""],""Совм...",...,Расходник для печати,Картридж лазерный Комус 729 (4368B002) пурпурн...,1,0.94,1,0.94,1,0.95,1,0.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306535,0.0,810680230,820119986,"Комплект 4 шт, Картридж лазерный NV Print 4072...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Расходн...",[черный],None,"[[-0.0944288, 0.8932959, 0.25614977, -0.191978...","[-0.6026151, 0.3903093, 0.74605453, -0.6013487...","{""Емкость"":[""Стандартная""],""Совместимые модели...",...,Расходник для печати,"Комплект 4 шт, Картридж лазерный NV Print 4072...",0,0.97,0,0.97,0,0.96,0,0.97
306536,0.0,812434186,815345877,Для iPhone 7 плюс 8 плюс военная броня гибридн...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Чехол"",...",[желтый],"[[0.28905904, -2.2125282, -0.51811993, 0.59951...","[[0.17367405, -1.9180702, -0.3562176, 1.175615...","[-0.73866224, 0.50003415, 0.38444322, -0.50629...","{""Вид чехла"":[""Обложка""],""Рекомендовано для"":[...",...,Чехол,Для iPhone 7 плюс 8 плюс военная броня гибридн...,1,0.50,0,0.87,0,0.52,1,0.65
306537,0.0,815637954,817550808,Карта памяти 64 ГБ,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Карты п...",None,"[[0.025763443, 0.29543704, -0.41198447, 0.1375...","[[0.19128616, 0.530021, 0.57061875, 0.59129846...","[-0.41885406, 0.4686782, 0.48758537, -0.452275...","{""Материал корпуса"":[""ABS пластик""],""Тип карты...",...,Карты памяти и флешки,Карта памяти 64 ГБ[SEP]Карта памяти 1 ТБ,0,0.97,0,0.97,0,0.98,0,0.98
306538,0.0,817327230,822083612,"Смартфон Mate48 Pro.. 8/256 ГБ, зеленый","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Смартфо...",[зелены

In [5]:
features.shape

(306540, 33)

In [6]:
from collections import Counter
# Сделал на трейне, хотя так лучше делать, но мало времени уже
features["json1"] = features["characteristic_attributes_mappin8g1"].apply(
    lambda x: json.loads(x) if x is not None else None)
features["json2"] = features["characteristic_attributes_mapping2"].apply(
    lambda x: json.loads(x) if x is not None else None)

KeyError: 'characteristic_attributes_mappin8g1'

In [ ]:
from pandas import json_normalize

In [ ]:
key_counter = Counter()
for col in ("json1", "json2"):
    for d in features[col].values:
        if type(d) is not dict:
            continue
        for k in d.keys():
            key_counter[k] += 1
            
popular_characteristics = {k[0] for k in key_counter.most_common(100)}

In [ ]:
key_counter = {k: v for k, v in key_counter.items() if k in popular_characteristics}

In [ ]:
for col in key_counter:
    features[col + "_1"] = ""
    features[col + "_2"] = ""

In [ ]:
key_dict_1 = {k: list() for k in key_counter}
for d in features["json1"].values:
    for k in key_dict_1:
        if type(d) != dict:
            key_dict_1[k].append("")
        elif k in d:
            key_dict_1[k].append(d[k])
        else:
            key_dict_1[k].append("")
for k, v in key_dict_1.items():
    features[k + "_1"] = ["; ".join(l) for l in v]

In [ ]:
key_dict_2 = {k: list() for k in key_counter}
for d in features["json2"].values:
    for k in key_dict_2:
        if type(d) != dict:
            key_dict_2[k].append("")
        elif k in d:
            key_dict_2[k].append(d[k])
        else:
            key_dict_2[k].append("")
for k, v in key_dict_2.items():
    features[k + "_2"] = ["; ".join(l) for l in v]

In [ ]:
features[features["Тип_1"] != features["Тип_2"]]

In [ ]:
import py_stringmatching as sm

In [ ]:
from py_stringmatching import QgramTokenizer

In [ ]:
def Levenshtein_distance(x, y):
    return sm.Levenshtein().get_sim_score(x, y)

features['Levenshtein_distance'] = features.apply(lambda row: Levenshtein_distance(row['name1'], row['name2']), axis=1)

In [ ]:
qg2_tok = QgramTokenizer(qval=2, return_set=True)

def Jaccard_distance(x, y):
    x_tokens = qg2_tok.tokenize(x)
    y_tokens = qg2_tok.tokenize(y)
    return sm.Jaccard().get_sim_score(x_tokens, y_tokens)

features['Jaccard_distance'] = features.apply(lambda row: Jaccard_distance(row['name1'], row['name2']), axis=1)

In [ ]:
features['Levenshtein_distance_types'] = features.apply(
    lambda row: Levenshtein_distance(str(row['Тип_1']), str(row['Тип_2'])), axis=1)
features['Levenshtein_distance_brands'] = features.apply(
    lambda row: Levenshtein_distance(str(row['Бренд_1']), str(row['Бренд_2'])), axis=1)

In [ ]:
features['Jaccard_distance_types'] = features.apply(
    lambda row: Jaccard_distance(row['Тип_1'], row['Тип_2']), axis=1)
features['Jaccard_distance_brands'] = features.apply(
    lambda row: Jaccard_distance(row['Бренд_1'], row['Бренд_2']), axis=1)

### Metric

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc


def pr_auc_macro(
    target_df: pd.DataFrame,
    predictions_df: pd.DataFrame,
    prec_level: float = 0.75,
    cat_column: str = "cat3_grouped"
) -> float:

    df = target_df.merge(predictions_df, on=["variantid1", "variantid2"])

    y_true = df["target"]
    y_pred = df["scores"]
    categories = df[cat_column]

    weights = []
    pr_aucs = []

    unique_cats, counts = np.unique(categories, return_counts=True)

    # calculate metric for each big category
    for i, category in enumerate(unique_cats):
        # take just a certain category
        cat_idx = np.where(categories == category)[0]
        y_pred_cat = y_pred[cat_idx]
        y_true_cat = y_true[cat_idx]

        # if there is no matches in the category then PRAUC=0
        if sum(y_true_cat) == 0:
            pr_aucs.append(0)
            weights.append(counts[i] / len(categories))
            continue
        
        # get coordinates (x, y) for (recall, precision) of PR-curve
        y, x, _ = precision_recall_curve(y_true_cat, y_pred_cat)
        
        # reverse the lists so that x's are in ascending order (left to right)
        y = y[::-1]
        x = x[::-1]
        
        # get indices for x-coordinate (recall) where y-coordinate (precision) 
        # is higher than precision level (75% for our task)
        good_idx = np.where(y >= prec_level)[0]
        
        # if there are more than one such x's (at least one is always there, 
        # it's x=0 (recall=0)) we get a grid from x=0, to the rightest x 
        # with acceptable precision
        if len(good_idx) > 1:
            gt_prec_level_idx = np.arange(0, good_idx[-1] + 1)
        # if there is only one such x, then we have zeros in the top scores 
        # and the curve simply goes down sharply at x=0 and does not rise 
        # above the required precision: PRAUC=0
        else:
            pr_aucs.append(0)
            weights.append(counts[i] / len(categories))
            continue
        
        # calculate category weight anyway
        weights.append(counts[i] / len(categories))
        # calculate PRAUC for all points where the rightest x 
        # still has required precision 
        try:
            pr_auc_prec_level = auc(x[gt_prec_level_idx], y[gt_prec_level_idx])
            if not np.isnan(pr_auc_prec_level):
                pr_aucs.append(pr_auc_prec_level)
        except ValueError:
            pr_aucs.append(0)
            
    return np.average(pr_aucs, weights=weights)

Choose features

In [ ]:
feats = ["pic_dist_0_perc", "pic_dist_25_perc", "pic_dist_50_perc", 
         "euclidean_name_bert_dist", "cosine_name_bert_dist",
        ]
text_features = ["name1", "name2"]

In [ ]:
feats += ["Levenshtein_distance", 
         "Jaccard_distance", 'Levenshtein_distance_types',
          'Levenshtein_distance_brands', 'Jaccard_distance_types', 'Jaccard_distance_brands']

In [ ]:
feats += ['deberta_label_0', 'deberta_score_0', 'deberta_label_1',
       'deberta_score_1', 'deberta_label_2', 'deberta_score_2',
       'deberta_label_4', 'deberta_score_4']

In [ ]:
key_features = [k + "_1" for k in key_counter.keys()]
key_features += [k + "_2" for k in key_counter.keys()]

In [ ]:
text_features = ["name1", "name2"]

In [ ]:
feats += key_features

In [ ]:
feats += text_features

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
splits = list(kfold.split(features))

Split dataset

In [ ]:
features["diff_variant_id"] = features["variantid2"] - features["variantid1"]

In [ ]:
import seaborn as sns
sns.displot(data=features, x="diff_variant_id", hue="target", bins=20, height=5, aspect=6/4)

In [ ]:
feats += ["diff_variant_id"]

In [ ]:
features.to_parquet("denis_features_3.parquet")

In [ ]:
catboost_params = {
    'iterations': 100000,
    'learning_rate': 0.01,
    "loss_function": 'CrossEntropy',
    'verbose': 100,
    "task_type": "GPU",
    "devices": '0',
    'early_stopping_rounds': 50,
    'random_seed': 42, 
    'depth': 6
}

pr_aucs = []

models = []

for split_i, split in enumerate(splits):
    split_features = features.copy()
    split_feats = feats.copy()
    split_feats = [s for s in split_feats if "deberta_" not in s or ("deberta_" in s and str(split_i) in s)]
    split_features = split_features[split_feats + ["target", "variantid1", "variantid2", "cat3_grouped"]]
    X_train = split_features.iloc[split[0]]
    X_test = split_features.iloc[split[1]]
    X_train, X_val = train_test_split(
        X_train, 
        test_size=0.1, random_state=42,
    #     stratify=X_train[["target", "cat3_grouped"]]
    )

#     X_train.to_parquet(f"/root/fold_{split_i}_train.parquet")
#     X_test.to_parquet(f"/root/fold_{split_i}_test.parquet")
#     X_val.to_parquet(f"/root/fold_{split_i}_val.parquet")

    y_test = X_test[["target", "variantid1", "variantid2"]]
    X_test = X_test.drop(["target"], axis=1)

    y_train = X_train["target"]
    y_val = X_val["target"]

    X_train = X_train.drop(["target"], axis=1)
    X_val = X_val.drop(["target"], axis=1)

    #split_feats = [f for f in split_feats if f not in  ("target")]
    train_pool = Pool(
        data=X_train[split_feats],
        label=y_train,
        text_features = text_features,
        cat_features=key_features
    )
    eval_pool = Pool(
        data=X_val[split_feats],
        label=y_val,
        text_features = text_features,
        cat_features=key_features
    )
    model = CatBoostClassifier(**catboost_params)

    model.fit(
        train_pool,
        eval_set=eval_pool,
        plot=True,
        verbose=True,
        use_best_model=True,
        early_stopping_rounds=50,
        metric_period=100
    )
    model.save_model(f"baseline_denis_3{split_i}.cbm")
    models.append(model)
    
    
    X_test["scores"] = model.predict_proba(X_test[split_feats])[:, 1]
    pr_auc_macro_metr = pr_auc_macro(
        target_df=y_test, 
        predictions_df=X_test,
        prec_level=0.75,
        cat_column="cat3_grouped"
    )
    
    print(dict(zip(model.feature_names_, model.feature_importances_)))
    print(f"pr_auc_macro_metr: {split_i}", pr_auc_macro_metr)
    
    precision, recall, thrs = precision_recall_curve(y_test["target"], X_test["scores"])
    pr_auc = auc(recall, precision)
    pr_aucs.append(pr_auc)
    fig, ax1 = plt.subplots(1, figsize=(15, 7))

    ax1.plot(recall, precision)
    ax1.axhline(y=0.75, color='grey', linestyle='-');

### Submission

In [ ]:
test_pairs = pd.read_parquet("hackathon_files_for_participants_ozon/test_pairs_wo_target.parquet")
test_etl = pd.read_parquet("hackathon_files_for_participants_ozon/test_data.parquet")

Generate the same features as for train

In [ ]:
test_features = (
    test_pairs
    .merge(
        test_etl
        .add_suffix('1'),
        on="variantid1"
    )
    .merge(
        test_etl
        .add_suffix('2'),
        on="variantid2"
    )
)

In [ ]:
test_features[["pic_dist_0_perc", "pic_dist_25_perc", "pic_dist_50_perc"]] = (
    test_features[["main_pic_embeddings_resnet_v11", "main_pic_embeddings_resnet_v12"]].apply(
        lambda x: pd.Series(get_pic_features_func(*x)), axis=1
    )
)

test_features[["euclidean_name_bert_dist", "cosine_name_bert_dist"]] = (
    test_features[["name_bert_641", "name_bert_642"]].apply(
        lambda x: pd.Series(text_dense_distances(*x)), axis=1
    )
)

In [ ]:
test_features["cat3"] = test_features["categories1"].apply(lambda x: json.loads(x)["3"])

In [ ]:
test_features.shape

In [ ]:
test_cat3_counts = test_features["cat3"].value_counts().to_dict()

cntr = 0
for cat3 in test_cat3_counts:
    if test_cat3_counts[cat3] < 50:
        cntr += test_cat3_counts[cat3]
        
cntr

In [ ]:
test_features["cat3_grouped"] = test_features["cat3"].apply(lambda x: x if test_cat3_counts[x] > 50 else "rest")
# btw you can rename to `rest` the same categories that were assigned to this group in train

In [ ]:
test_features["text"] = test_features["name1"] + "[SEP]" + test_features["name2"]

In [ ]:
%%time
for i in range(5):
    print(i)
    # the model for the 3rd fold sucks
    # no time to retrain
    if i == 3:
        continue
    pipe = pipeline("text-classification", f"/predefined_classes_best_{i}", device=0, batch_size=32)


    pipe_preds = pipe(list(test_features["text"].values), padding="longest", max_length=256, truncation=True)

    test_features[f"deberta_label_{i}"] = [p["label"] for p in pipe_preds]
    test_features[f"deberta_score_{i}"] = [round(p["score"], 2) for p in pipe_preds]

    test_features.to_parquet("denis_test_features.parquet")

    feats += [f"deberta_label_{i}", f"deberta_score_{i}"]
    
    del pipe
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
test_features["diff_variant_id"] = test_features["variantid2"] - test_features["variantid1"]

In [ ]:
for split_i, split in enumerate(splits):
    submission_example = test_features.copy()
    split_feats = feats.copy()
    split_feats = [s for s in split_feats if "deberta_" not in s or ("deberta_" in s and str(split_i) in s)]
    submission_example = submission_example[split_feats + ["variantid1", "variantid2", "cat3_grouped"]]
    test_features[f"target_{split_i}"] = models[split_i].predict_proba(submission_example[split_feats])[:, 1]

In [ ]:
((test_features["target_0"] +
 test_features["target_1"] +
 test_features["target_2"] +
#  test_features["target_3"] +
 test_features["target_4"]) / 4).sum()

In [ ]:
# submission_example["target"] = 
((test_features["target_0"] +
 test_features["target_1"] +
 test_features["target_2"] +
 test_features["target_3"] +
 test_features["target_4"]) / 5).sum()

In [ ]:
submission_example["target"] = ((test_features["target_0"] +
 test_features["target_1"] +
 test_features["target_2"] +
#  test_features["target_3"] +
 test_features["target_4"]) / 4)

In [ ]:
submission_example = submission_example[["variantid1", "variantid2", "target"]]

In [ ]:
submission_example.drop_duplicates().merge(
    test_features[["variantid1", "variantid2"]].drop_duplicates(["variantid1", "variantid2"]),
    on=["variantid1", "variantid2"]
).to_csv("submission_example.csv", index=False)

Upload your submission to leaderboard :)

### Recommendations 

- Work with names, in electronics names often contain a lot of useful information for matching.
- Don't forget about attributes: working with it will allow your model to better distinguish matches from non-matches.

In [ ]:
test_features["target"] = submission_example["target"]

In [ ]:
test_features[test_features["name1"] != test_features["name2"]]